## Import Modules

In [112]:
# import module
import pandas as pd
from tqdm.notebook import tqdm
import time
import numpy as np

## Read CSV

In [113]:
# read csv
df1 = pd.read_csv("delivery_orders_march.csv")

In [114]:
df1.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,1583138397,1.583385e+09,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,1583309968,1.583463e+09,1.583799e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,1583306434,1.583460e+09,NaN,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,1583419016,1.583556e+09,NaN,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,1583318305,1.583480e+09,NaN,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


## Convert epoch time to normal time format

In [115]:
df1[['pick','1st_deliver_attempt','2nd_deliver_attempt']] += 8*60*60 #to GMT+8
df1["pick"] = pd.to_datetime(df1['pick'],unit='s').dt.date
df1['1st_deliver_attempt'] = pd.to_datetime(df1['1st_deliver_attempt'],unit='s').dt.date
df1['2nd_deliver_attempt'] = df1['2nd_deliver_attempt'].replace(np.nan,0) 
df1['2nd_deliver_attempt'] = pd.to_datetime(df1['2nd_deliver_attempt'],unit='s').dt.date

In [116]:
df1.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,2020-03-02,2020-03-05,1970-01-01,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,2020-03-04,2020-03-06,2020-03-10,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,2020-03-04,2020-03-06,1970-01-01,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,2020-03-05,2020-03-07,1970-01-01,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,2020-03-04,2020-03-06,1970-01-01,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


## Determine sla for each rows based on the buyer and seller address

In [117]:
# function to return the preferred format for address
def transform(text):
    if "manila" in text.lower():
        return "Metro Manila"
    elif "luzon" in text.lower():
        return "Luzon"
    elif "visayas" in text.lower():
        return "Visayas"
    elif "mindanao" in text.lower():
        return "Mindanao"

# function to return the value of sla
def trans_value(buyer_address, seller_address):
    if (buyer_address == "Metro Manila") and (seller_address == "Metro Manila"):
        return 3
    elif ((buyer_address == "Metro Manila") and (seller_address == "Luzon")) or ((buyer_address == "Luzon") and (seller_address == "Metro Manila")) or ((buyer_address == "Luzon") and (seller_address == "Luzon")):
        return 5
    else: 
        return 7

In [118]:
sla_ = []

for ba, sa in tqdm(df1[['buyeraddress','selleraddress']].itertuples(index=False)):
    # ba = buyer address
    # sa = seller address
    
    ba = ba.split(",")[-1]
    sa = sa.split(",")[-1]
    
    ba = ba.split(" ")[-1]
    sa = sa.split(" ")[-1]
    
    ba = transform(ba)
    sa = transform(sa)
    
    sla = trans_value(ba, sa)
    sla_.append(sla)

df1["sla"] = sla_
df1 = df1.drop(columns=['buyeraddress', 'selleraddress'])

In [119]:
df1.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,sla
0,2215676524,2020-03-02,2020-03-05,1970-01-01,3
1,2219624609,2020-03-04,2020-03-06,2020-03-10,3
2,2220979489,2020-03-04,2020-03-06,1970-01-01,3
3,2221066352,2020-03-05,2020-03-07,1970-01-01,3
4,2222478803,2020-03-04,2020-03-06,1970-01-01,5


## Determine if the delivery is late or not

In [120]:
public_holiday = ['2020-03-08','2020-03-25','2020-03-30','2020-03-31']

# count the time difference between pickup time and 1st deliver attempt
df1['1st_pick'] = np.busday_count(df1['pick'], df1['1st_deliver_attempt'], weekmask='1111110', holidays=public_holiday)

# count the time difference between 1st deliver attempt and 2nd deliver attempt
df1['2nd_pick'] = np.busday_count(df1['1st_deliver_attempt'], df1['2nd_deliver_attempt'], weekmask='1111110', holidays=public_holiday)

# determine if it is late or not
df1['is_late'] = (df1['1st_pick'] > df1['sla']) | (df1['2nd_pick'] > 3)

In [121]:
df1.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,sla,1st_pick,2nd_pick,is_late
0,2215676524,2020-03-02,2020-03-05,1970-01-01,3,3,-15708,False
1,2219624609,2020-03-04,2020-03-06,2020-03-10,3,2,3,False
2,2220979489,2020-03-04,2020-03-06,1970-01-01,3,2,-15709,False
3,2221066352,2020-03-05,2020-03-07,1970-01-01,3,2,-15710,False
4,2222478803,2020-03-04,2020-03-06,1970-01-01,5,2,-15709,False


## Create new Dataframe

In [122]:
df = pd.DataFrame({'orderid':df1['orderid'], 'is_late':df1['is_late'].apply(int)})

In [123]:
df.head()

,orderid,is_late
0,2215676524,0
1,2219624609,0
2,2220979489,0
3,2221066352,0
4,2222478803,0


## Export to CSV

In [124]:
# write to csv
df.to_csv("answer.csv", header = True, index=False)